### Test Downloads

In [1]:
from data_pipeline.src.downloader import run_downloads

run_downloads()


Saving extracted PBMC datasets to: /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw

⇩ Downloading and extracting to /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD34


CD34: 100%|██████████| 38.0M/38.0M [00:00<00:00, 56.6MB/s]


✓ Extracted to /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD34
✓ Finished CD34
------------------------------------------------------------
⇩ Downloading and extracting to /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD19_B


CD19_B: 100%|██████████| 18.0M/18.0M [00:00<00:00, 44.9MB/s]


✓ Extracted to /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD19_B
✓ Finished CD19_B
------------------------------------------------------------
⇩ Downloading and extracting to /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD56_NK


...

✓ Extracted to /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD4_CD45RO
✓ Finished CD4_CD45RO
------------------------------------------------------------
⇩ Downloading and extracting to /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8


CD8: 100%|██████████| 20.0M/20.0M [00:00<00:00, 49.2MB/s]


✓ Extracted to /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8
✓ Finished CD8
------------------------------------------------------------
⇩ Downloading and extracting to /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8_CD45RA


CD8_CD45RA: 100%|██████████| 20.9M/20.9M [00:00<00:00, 47.3MB/s]


✓ Extracted to /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8_CD45RA
✓ Finished CD8_CD45RA
------------------------------------------------------------
All PBMC datasets ready in: /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw


In [2]:
from data_pipeline.src.config import DATASETS
from data_pipeline.src.load_anndata import load_anndata
from pathlib import Path

folder = Path("data_pipeline/data/pbmc_raw/CD4_CD45RO")
meta = DATASETS["CD4_CD45RO"]

adata = load_anndata(folder, meta)
adata

Loading 10x data from: data_pipeline/data/pbmc_raw/CD4_CD45RO/filtered_matrices_mex/hg19
✓ Loaded 10224 cells × 32738 genes from CD4_CD45RO


AnnData object with n_obs × n_vars = 10224 × 32738
    obs: 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'cell_type_lvl4', 'dataset', 'cell_type_lvl1__code', 'cell_type_lvl2__code', 'cell_type_lvl3__code', 'cell_type_lvl4__code'
    var: 'gene_ids'

In [2]:
from data_pipeline.src.combine import run_combine

combined, collection = run_combine(
    do_write_shards=True,
    write_combined=True,
    harmonize_var=False
)

Loading all PBMC AnnData objects...
📦 Loading dataset: CD34
Loading 10x data from: /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD34/filtered_matrices_mex/hg19
✓ Loaded 9232 cells × 32738 genes from CD34
📦 Loading dataset: CD19_B
Loading 10x data from: /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD19_B/filtered_matrices_mex/hg19
✓ Loaded 10085 cells × 32738 genes from CD19_B
📦 Loading dataset: CD56_NK
Loading 10x data from: /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD56_NK/filtered_matrices_mex/hg19
✓ Loaded 8385 cells × 32738 genes from CD56_NK
📦 Loading dataset: CD4_helper
Loading 10x data from: /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD4_helper/filtered_matrices_mex/hg19
✓ Loaded 11213 cells × 32738 genes from CD4_helper
📦 Loading dataset: CD4_CD25
Loading 10x data from: /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipe

/Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/.venv/lib/python3.12/site-packages/anndata/_core/anndata.py:1796: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


✓ Combined AnnData written to /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_processed/pbmc_combined.h5ad
✓ Label maps saved to /Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/data_pipeline/data/pbmc_processed/label_maps.json
Building in‑memory AnnCollection...


/Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/.venv/lib/python3.12/site-packages/legacy_api_wrap/__init__.py:88: UserWarning: Observation names are not unique.
  return fn(*args_all, **kw)


✓ AnnCollection constructed from 9 shards.

Combine stage complete!



In [3]:
import scanpy as sc
adata_combined = sc.read_h5ad("data_pipeline/data/pbmc_processed/pbmc_combined.h5ad")
adata_combined

/Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/.venv/lib/python3.12/site-packages/anndata/_core/anndata.py:1796: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 92043 × 32738
    obs: 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'cell_type_lvl4', 'dataset', 'cell_type_lvl1__code', 'cell_type_lvl2__code', 'cell_type_lvl3__code', 'cell_type_lvl4__code'
    var: 'gene_ids'
    uns: 'label_maps'

In [1]:
from data_pipeline.src.dataloader import build_dataloaders

train_loader, val_loader, test_loader = build_dataloaders(
    shard_dir="data_pipeline/data/pbmc_processed/shards",
    label_maps_path="data_pipeline/data/pbmc_processed/label_maps.json",
    batch_size=512,
    one_hot=True,      
)

for i, batch in enumerate(train_loader):
        print("Example batch shapes:")
        print("X:", batch["X"].shape, "y1:", batch["y1"].shape, "y4:", batch["y4"].shape)
        break

/Users/tiffneyaina/scVAE_mva2025_fresh/scVAE_mva2025/.venv/lib/python3.12/site-packages/legacy_api_wrap/__init__.py:88: UserWarning: Observation names are not unique.
  return fn(*args_all, **kw)


✓ Built AnnCollection with 92043 total cells.
Split: train=73634, val=9204, test=9205
✓ Train=73634, Val=9204, Test=9205
Example batch shapes:
X: torch.Size([512, 1, 32738]) y1: torch.Size([512, 1, 2]) y4: torch.Size([512, 1, 9])
